# Cohort Analysis & Customer Segmentation (RFM + K-Means)

**Ready-to-run Jupyter Notebook**

This notebook walks you step-by-step through:

1. Loading and cleaning an e-commerce transactions dataset
2. Computing RFM (Recency, Frequency, Monetary) scores
3. Performing K-Means clustering on RFM features
4. Performing Cohort Analysis and plotting a retention heatmap
5. Visualizing segments and extracting business insights

> **Note:** This notebook expects a dataset with columns similar to the `OnlineRetail.csv` dataset (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country).

If you don't have the dataset locally, you can download the **Online Retail** dataset from Kaggle or other public sources and place it in the same folder as this notebook with the name `OnlineRetail.csv`.

---

Run cells in order. Explanations and comments are included inline.

In [ ]:
# Install required packages if not already installed.
# Uncomment and run if you need to install packages in your environment.
# !pip install pandas numpy matplotlib seaborn scikit-learn plotly openpyxl

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
import os

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
sns.set(style='whitegrid')

print("Libraries imported. Current working directory:", os.getcwd())

In [ ]:
# Load dataset. Replace the filename if yours is different.
# Expected filename: 'OnlineRetail.csv'
fname = 'OnlineRetail.csv'

if not os.path.exists(fname):
    print(f"File '{fname}' not found in the current directory.")
    print("Please download the Online Retail dataset (or your transactions CSV) and place it here with the name 'OnlineRetail.csv'.")
    print("You can also change the variable `fname` to the correct path.")
else:
    df = pd.read_csv(fname, encoding='latin1')
    print("Dataset loaded. Shape:", df.shape)
    display(df.head())

In [ ]:
# Basic cleaning and preprocessing
# This block will run only if 'df' exists (i.e., dataset was loaded).

if 'df' in globals():
    # Show column names and dtypes
    print("Columns:", df.columns.tolist())
    print(df.dtypes)

    # Remove rows without CustomerID (we need customer-level analysis)
    initial_len = len(df)
    df = df.dropna(subset=['CustomerID'])
    print(f"Dropped rows without CustomerID: {initial_len - len(df)} rows removed. New shape: {df.shape}")

    # Remove canceled transactions (InvoiceNo starting with 'C' often marks cancellations)
    if df['InvoiceNo'].dtype == object:
        df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

    # Create TotalPrice column (Quantity * UnitPrice)
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0)
    df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce').fillna(0.0)
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

    # Convert InvoiceDate to datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

    # Drop rows with non-positive TotalPrice (optional)
    df = df[df['TotalPrice'] > 0]

    print("After cleaning, dataset shape:", df.shape)
    display(df[['InvoiceNo','InvoiceDate','CustomerID','Quantity','UnitPrice','TotalPrice']].head())

In [ ]:
# Compute RFM metrics
if 'df' in globals():
    # Latest order date (reference date for recency). Use a fixed date or dataset max date.
    latest_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
    print("Reference date for Recency (latest_date):", latest_date.date())

    # Aggregate per customer
    rfm = df.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (latest_date - x.max()).days,  # Recency in days
        'InvoiceNo': 'nunique',  # Frequency = number of orders/invoices
        'TotalPrice': 'sum'  # Monetary = total spent
    }).reset_index()

    rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
    display(rfm.describe().T)
    display(rfm.head())

In [ ]:
# Assign RFM scores using quantiles (1-5)
if 'rfm' in globals():
    # We convert Recency so that lower recency (recent purchase) gets higher score.
    r_labels = [5,4,3,2,1]  # 5 = most recent, 1 = least recent
    rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=r_labels).astype(int)

    f_labels = [1,2,3,4,5]  # higher frequency => higher score
    rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=5, labels=f_labels).astype(int)

    m_labels = [1,2,3,4,5]  # higher monetary => higher score
    rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=5, labels=m_labels).astype(int)

    rfm['RFM_Score'] = rfm['R_Score']*100 + rfm['F_Score']*10 + rfm['M_Score']  # formatted score
    rfm['RFM_Sum'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

    display(rfm.head())
    print("Value counts for RFM Sum:")
    display(rfm['RFM_Sum'].value_counts().sort_index())

In [ ]:
# K-Means clustering on RFM numeric values
if 'rfm' in globals():
    # Prepare features
    features = rfm[['Recency','Frequency','Monetary']].copy()

    # It's recommended to log-transform Monetary to reduce skew
    features['Monetary_log'] = np.log1p(features['Monetary'])
    features_for_clustering = features[['Recency','Frequency','Monetary_log']]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features_for_clustering)

    # Choose number of clusters (k). Here we use k=4 as an example.
    k = 4
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(X_scaled)

    rfm['Cluster'] = clusters
    print("Cluster sizes:")
    display(rfm['Cluster'].value_counts().sort_index())

    # Show cluster centers (inverse transform to original scale for interpretation)
    centers = scaler.inverse_transform(kmeans.cluster_centers_)
    centers_df = pd.DataFrame(centers, columns=features_for_clustering.columns)
    display(centers_df)

In [ ]:
# Analyze clusters: mean RFM per cluster
if 'rfm' in globals():
    cluster_analysis = rfm.groupby('Cluster').agg({
        'Recency':'mean',
        'Frequency':'mean',
        'Monetary':'mean',
        'RFM_Sum':'mean',
        'CustomerID':'count'
    }).rename(columns={'CustomerID':'NumCustomers'}).reset_index()
    display(cluster_analysis)

    # Visualize clusters using scatter plot (Recency vs Monetary) with Frequency as size
    fig = px.scatter(rfm, x='Recency', y='Monetary', color='Cluster', size='Frequency',
                     title='Customer Segments: Recency vs Monetary (size=Frequency)',
                     hover_data=['CustomerID','RFM_Sum'])
    fig.show()

In [ ]:
# Cohort Analysis: calculate retention table
if 'df' in globals():
    # Create Month columns for order and cohort
    df['OrderMonth'] = df['InvoiceDate'].dt.to_period('M')
    df['CohortMonth'] = df.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')

    # CohortIndex: number of months since the cohort start
    def get_month_diff(end, start):
        return (end.year - start.year) * 12 + (end.month - start.month)

    df['CohortIndex'] = df.apply(lambda row: get_month_diff(row['OrderMonth'].to_timestamp(), row['CohortMonth'].to_timestamp()), axis=1)

    cohort_data = df.groupby(['CohortMonth','CohortIndex'])['CustomerID'].nunique().reset_index()
    cohort_pivot = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

    # Cohort size (number of users in first month)
    cohort_size = cohort_pivot.iloc[:,0]
    retention = cohort_pivot.divide(cohort_size, axis=0)

    # Display retention heatmap
    plt.figure(figsize=(12,8))
    sns.heatmap(retention, annot=True, fmt='.0%', cmap='Blues')
    plt.title('Cohort Retention Table')
    plt.ylabel('Cohort Month')
    plt.xlabel('Months Since Cohort')
    plt.show()

    display(retention.head())

## Business Insights & Next Steps

- **Interpret clusters**: Label clusters as "High-Value", "At-Risk", "New", "Low-Value" based on means in `cluster_analysis`.
- **Actions**:
  - High-value: loyalty programs, exclusive offers.
  - At-risk: re-engagement campaigns, special discounts.
  - New: welcome series and onboarding.
  - Low-value: evaluate cost-to-serve; tailor offerings.
- **Next steps for production**:
  - Store RFM scores and cluster IDs in a database for downstream use.
  - Build a dashboard (Power BI / Tableau / Streamlit) to track cohort retention and segment distribution.
  - Consider predictive models (churn prediction) using time-series or classification models.

---

If you want, I can:
1. Generate a downloadable copy of this notebook.
2. Create a small sample CSV to test the notebook if you don't have the real dataset.
3. Convert key plots into static PNGs for embedding in a report.